In [2]:
'''Construction of the output file, from all the small output obtained'''

'Construction of the output file, from all the small output obtained'

In [3]:
import csv
import heapq

In [4]:
folder='D:/IA/Kaggle/Facebook/'

In [5]:
def extract_sub_parts(name, output_names=None):
    '''extract sub files of name (break when the first column restart at a lower id)
    
    not beautiful, don't take the exception of the writter...
    '''
    if not output_names :
        output_names=name[:-4]+'_'
    with open(name,newline='') as f:
        reader = csv.reader(f)
        n=0
        with open(output_names+str(n)+'.csv','w',newline='') as file:
            writer = csv.writer(file)
            current_line = 0
            for line in reader :
                if int(line[0]) < current_line :
                    file.close()
                    n+=1
                    file=open(output_names+str(n)+'.csv','w',newline='')
                    writer = csv.writer(file)
                writer.writerow(line)
                current_line = int(line[0])



In [6]:
def merge_files(names_input, name_output):
    with control_managers([open(n, newline='') for n in names_input]) as files, open(name_output,'w',newline='') as f :
        readers = [csv.reader(fi) for fi in files]
        writer = csv.writer(f)
        values = [(int(line[0]), line, i) for i,line in enumerate([r.__next__() for r in readers])]
        heapq.heapify(values)
        new_line = None
        while values :
            line = heapq.heappushpop(values, new_line) if new_line else heapq.heappop(values)
            writer.writerow(line[1])
            new_line=None
            for next_line in readers[line[2]] : #used as a try/else/finally for StopIteration
                new_line = (int(next_line[0]), next_line,line[2])
                break
    
    
class control_managers():
    def __init__(self, managers_list):
        self.managers = managers_list
    def __enter__(self):
        return [m.__enter__() for m in self.managers]
    def __exit__(self, type, value, traceback):
        [m.__exit__() for m in self.managers]

In [21]:
def check_and_fit(folder, sample, results, output):
    """check the size of the results and had head of sample, in output"""
    with open(folder+sample, newline='') as sf, open(folder+results, newline='') as rf, open(folder+output,'w', newline='') as wf :
        s, r, w = csv.reader(sf), csv.reader(rf), csv.writer(wf)
        w.writerow(s.__next__())
        sizes=[1,0,1]
        for line in r :
            w.writerow(line)
            sizes[1]+=1
            sizes[2]+=1
            try :
                s.__next__()
                sizes[0]+=1
            except StopIteration :
                print('sample already finished')
                print(line)
        for line in s:
            sizes[0]+=1
            print('sample not finished')
            print(line)
        print(sizes)

In [36]:
def check_missing_lines(folder, results):
    '''check what lines are missing'''
    out=[]
    with open(folder+results, newline='') as file:
        reader=csv.reader(file)
        current = -1
        reader.__next__()
        for line in reader :
            current += 1
            if int(line[0]) != current :
                out.append(current)
                current=int(line[0])
    return out
            

In [32]:
def collect_lines(folder, test, lines):
    out=[]
    with open(folder+test,newline='') as file:
        reader = csv.reader(file)
        reader.__next__()
        nextl=lines.pop(0)
        for line in reader :
            if int(line[0]) == nextl :
                out.append(line)
                try :
                    nextl=lines.pop(0)
                except Error :
                    nextl=0
    return out


In [34]:
def complete_and_fit(folder, sample, results, output):
    """let white row for forgot values"""
    with open(folder+sample, newline='') as sf, open(folder+results, newline='') as rf, open(folder+output,'w', newline='') as wf :
        s, r, w = csv.reader(sf), csv.reader(rf), csv.writer(wf)
        w.writerow(s.__next__())
        sizes=[1,0,1]
        next_complete_line=r.__next__()
        for line in s :
            if next_complete_line and int(line[0]) == int(next_complete_line[0]):#we have the good line
                w.writerow(next_complete_line)
                try :
                    next_complete_line = r.__next__()
                except StopIteration :
                    next_complete_line = None
            else :
                w.writerow(line)

In [55]:
def extract_regular_lines(folder,file,N=100000,N_lines=100000):
    with open(folder+file,newline='') as f:
        r=csv.reader(f)
        n=0
        for line in r:
            if n%N==0:
                print(line)
                n=0
                N_lines-=1
                if N_lines==0 :
                    return None
            n+=1
    

In [8]:
#execution

In [12]:
extract_sub_parts(folder+'output3.csv')

In [16]:
merge_files([folder+'output'+str(j)+'_'+str(i)+'.csv' for i in range(20) for j in range(20)],folder+'outputGLOBAL.csv')

In [35]:
complete_and_fit(folder,'sample_submission.csv','outputGLOBAL.csv','Pibiflex_submission.csv')

In [39]:
lines=check_missing_lines(folder, 'outputGLOBAL.csv')

In [40]:
print(len(lines))

3314


In [41]:
lines_values=collect_lines(folder,'test.csv',lines)

KeyboardInterrupt: 

In [ ]:
position=[(x[1],x[2] for x in lines_values)]
position=list(zip(position))
fig = plt.figure(figsize=(20, 20))
ax = fig.add_subplot(111)#, projection='3d')
ax.scatter(position[0],position[1],linewidths=0)

In [59]:
extract_regular_lines(folder, 'test.csv',N=1,N_lines=50)

['row_id', 'x', 'y', 'accuracy', 'time']
['0', '0.1675', '1.3608', '107', '930883']
['1', '7.3909', '2.5301', '35', '893017']
['2', '8.0978', '2.3473', '62', '976933']
['3', '0.999', '1.0591', '62', '907285']
['4', '0.667', '9.7254', '40', '914399']
['5', '0.1771', '0.0022', '161', '814077']
['6', '5.5299', '4.6581', '2', '930759']
['7', '8.6021', '3.1744', '4', '862115']
['8', '4.225', '6.3435', '162', '787391']
['9', '0.6489', '6.2611', '39', '793166']
['10', '2.7546', '6.6373', '315', '807936']
['11', '0.9998', '7.4121', '80', '913991']
['12', '4.9477', '6.4959', '7', '938599']
['13', '5.0336', '1.2666', '83', '800780']
['14', '0.948', '4.4362', '59', '892117']
['15', '0.2898', '3.0031', '57', '909801']
['16', '5.2386', '1.8262', '52', '867042']
['17', '2.9797', '7.7333', '170', '822650']
['18', '5.0956', '0.1904', '56', '841934']
['19', '2.7856', '1.7072', '61', '884954']
['20', '3.6321', '2.2853', '67', '905857']
['21', '9.9039', '5.9419', '76', '842668']
['22', '5.5745', '3.4077'

In [48]:
extract_regular_lines(folder, 'sample_submission.csv')

['row_id', 'place_id']
['99999', '7041827533 7404370151 9858681735']
['199999', '9794813531 1094723460 1358471000']
['299999', '1307322953 9994236123 9003822709']
['399999', '4521095393 4356523554 1939736249']
['499999', '1227752784 1359167240 7920649209']
['599999', '5484587131 7767830704 3407767346']
['699999', '1010496931 4558402323 1443591053']
['799999', '5449188393 3973352211 3231148897']
['899999', '7498991789 2042110820 4139794973']
['999999', '3885273378 4719146568 9267626150']
['1099999', '3410887515 9084690864 4363987023']
['1199999', '1608186657 7184911171 3954331366']
['1299999', '5253097169 7809835411 7739549317']
['1399999', '2462676557 9018937828 4117797522']
['1499999', '1725755607 9804704950 4041640586']
['1599999', '7212742750 6523625623 7512623630']
['1699999', '5961278997 8228310043 8814028353']
['1799999', '4430439961 5721681234 1941824359']
['1899999', '7457424654 3869576986 1017703558']
['1999999', '3886717477 2209775329 7641331421']
['2099999', '3166188869 6255

In [49]:
a=[2,5,4,6,8]

In [50]:
b=sorted(a)

In [53]:
b.pop()

5